# Pandas Performance

I recently ported a program based on `list`s and `dicts` of instances of some container classes to use `Pandas.DataFrame`s. While the new code was terser and saver, it was to my surprise much slower. Improving performance by vectorization was not everywhere an option, but I could improve the situation by optimizing lookups. Here are some `%timeit` stats, which should be taken with a grain of salt because results might get cached.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame(np.random.rand(10000, 5), columns=[i for i in 'abcde'])
lst = [{i: row[i] for i in 'abcde'} for _, row in df.iterrows()]

%timeit pd.DataFrame(np.random.rand(10000, 5), columns=[i for i in 'abcde'])
%timeit [{i: row[i] for i in 'abcde'} for _, row in df.iterrows()]

712 µs ± 12.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
479 ms ± 25.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
def vec_add(df):
    df['b'] + df['c']
    
%timeit vec_add(df)

87.7 µs ± 7.32 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [4]:
def iter_lst(lst):
    for row in lst:
        row['b'] + row['c']

%timeit iter_lst(lst)

1.74 ms ± 119 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
def iter_tuples(df):
    for t in df.itertuples():
        t.b + t.c

%timeit iter_tuples(df)

7.16 ms ± 1.09 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
def iter_rows(df):
    for _, row in df.iterrows():
        row['b'] + row['c']

%timeit iter_rows(df)

343 ms ± 41.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%timeit df['b'][3]
%timeit df['b'].iloc[3]
%timeit df.loc[3, 'b']
%timeit df.loc[3]['b']
%timeit df.iloc[3, 1]
%timeit df.iloc[3]['b']

11 µs ± 1.6 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
10.8 µs ± 647 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
6.04 µs ± 181 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
79.1 µs ± 1.66 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
6.84 µs ± 110 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
74 µs ± 12 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [16]:
%timeit df.values[0:5, 1]
%timeit df['b'].values[0:5]
%timeit df['b'].iloc[0:5]
%timeit df.iloc[0:5, 1]
%timeit df.iloc[0:5]['b']

4.25 µs ± 46.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.95 µs ± 649 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
32.3 µs ± 4.42 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
78.7 µs ± 1.56 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
92.1 µs ± 8.56 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
